#### IMPORTS

In [10]:
#Importa os módulos usados

import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from typing import Dict, List
import copy


# Define um tipo de dado similar ao Pascal "record" or C "struct"

class Struct:
    pass

#### Obj Solução

In [11]:
class Solucao:
    id: int = 0
    PAs_ativos: Dict[int, int] = {}
    PA_id_por_cliente: Dict[int, int] = {}
    fitness_PA_min: float = 0
    fitness_PA_min_penalizado: float = 0
    fitness_dist_min: float = 0
    fitness_dist_min_penalizado: float = 0
    porcentagem_CL_antendidos = 0
    nCL_por_PA: Dict[int, int] = {}

#### Solução Inicial

In [13]:
# heuristica

def heuristica_insercao_mais_proxima(clientes, num_pontos_acesso):
    sol = Solucao()

    # Calcular distância euclidiana entre clientes e pontos de acesso
    distancias = np.zeros((len(clientes), num_pontos_acesso))
    for i, cliente in enumerate(clientes):
        for j in range(num_pontos_acesso):
            distancias[i, j] = np.sqrt((cliente.x - j) ** 2 + (cliente.y - j) ** 2)

    # Atribuir cada cliente ao ponto de acesso mais próximo
    for j in range(len(clientes)):
        distancias_cliente = distancias[j]
        pa_mais_proximo = np.argmin(distancias_cliente)
        sol.PA_id_por_cliente[j] = pa_mais_proximo  # Dá o PA mais proximo como ativo para o cliente j
        sol.PAs_ativos[pa_mais_proximo] = 1  # Ativa o PA mais próximo

    return sol

# Solução inicial
def sol_inicial(probdata):
    
    sol = heuristica_insercao_mais_proxima(probdata.clientes, probdata.nPA)

    return sol

#### Funções UTIL para definição de dados

In [22]:
class Cliente:
    def __init__(self, x, y, consumo_banda):
        self.x = x
        self.y = y
        self.consumo_banda = consumo_banda

    def __repr__(self):
        return f"Cliente(x={self.x}, y={self.y}, consumo_banda={self.consumo_banda})"

# Função para ler os clientes do arquivo
def ler_clientes(nome_arquivo):
    clientes = []
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
        for linha in linhas:
            x, y, consumo_banda = map(float, linha.strip().split(','))
            cliente = Cliente(x, y, consumo_banda)
            clientes.append(cliente)
    return clientes

In [23]:
def clientProcessing(clientList, nCL=495, nPA=81**2):

    cons = np.zeros(nCL)
    dist = np.zeros((nCL, nPA))
    exp = np.zeros((nCL, nPA))

    cl_index = 0
    for cliente in clientList:
        cons[cl_index] = cliente.consumo_banda

        for PA_index in range(nPA):
            PA_x = (PA_index % 81) * 5
            PA_y = (PA_index // 81) * 5
            
            dist[cl_index, PA_index] = np.sqrt((cliente.x - PA_x)**2 + (cliente.y - PA_y)**2)
            #if PA_index % 81 == 80 & PA_index // 81 == 80:
            #    print(PA_x, end="-")
            #    print(PA_y, end=" to ")
            #    print(cliente.x, end="-")
            #    print(cliente.y, end=": ")
            #    print(dist[cl_index, PA_index])
            exp[cl_index, PA_index] = 1/dist[cl_index, PA_index]

        cl_index += 1

    return cons, dist, exp

#### Função definição de dados para o problema

In [12]:
def probdef(nPA=81**2, nCL=495):
    
    probdata = Struct()

    # Uso da função para ler os clientes do arquivo
    probdata.clientes = ler_clientes('data/clientes.csv')

    C, D, E = clientProcessing(probdata.clientes)

    probdata.CL_cons = C # importar consumo dos clientes do .csv
    probdata.dist_CL_PA = D # distância entre clientes e PAs
    probdata.exp_CL_PA = E # exposição dos clientes aos PAs

    probdata.PA_cap = 54 # em Mbps
    probdata.PA_raio = 84 # em metros
    probdata.CL_min_p = 0.05 # porcentagem minima de clientes
    probdata.exp_coef = 1 # coeficiente de exposição
    probdata.falloff = 1 # fator de decaimento

    probdata.nPA = nPA
    probdata.nCL = nCL
    probdata.nPA_max = 30

    return probdata

#### Função penalidades

In [24]:
def penalties(ponto_de_acesso_id: int, sol: Solucao, probdata: Struct):
    soma_do_consumo = 0

    for cliente in probdata.CL_list:
        # se o ponto ainda nao esta atendido
        cliente_nao_atendido = sol.porcentagem_CL_antendidos[cliente.id] == -1
        # se o ponto nao fara com que o ponto de acesso fique sobrecarregado com a banda necessária
        banda_suportada = (soma_do_consumo + cliente.consumo_de_banda) <= probdata.PA_cap
        # se o ponto esta dentro do alcance do PA
        cliente_dentro_do_range = probdata.dist_PA_CL[ponto_de_acesso_id][cliente.id] <= probdata.PA_raio
        
        if ( cliente_nao_atendido and banda_suportada and cliente_dentro_do_range ):
            cliente.id = ponto_de_acesso_id
            soma_do_consumo += cliente.consumo_de_banda
            sol.porcentagem_CL_antendidos[cliente.id] = ponto_de_acesso_id

#### Funções Objetivo

In [6]:
# Função objetivo 1: Minimizar número de PAs ativos
def fobj_minPA(x, y, probdata):

    sol = np.transpose(np.array(y.solution))

    y.fitness = np.sum(sol)
    y.penalidade = penalties(x, y, probdata)
    y.fitness_penalizado = y.fitness + y.penalidade

    return y


# Função objetivo 2: Minimizar distância cumulativa de clientes e PAs
def fobj_mindist(x, y, probdata):

    sol = x.solution

    fit_matrix = np.multiply(sol, probdata.dist_CL_PA)
    x.fitness = sum(fit_matrix)
    x.penalidade = penalties(x, y, probdata)
    x.fitness_penalizado = x.fitness + x.penalidade

    return x

#### Funções UTIL para VNS

In [8]:
# Shake implementation // TODO: MUDAR HEURISTICAS
def shake(sol: Solucao, k, probdata):
    
    y = copy.deepcopy(x)
    r = np.random.permutation(probdata.n)       
    
    if k == 1:             # apply not operator in one random position
        y.solution[r[0]] = not(y.solution[r[0]])
        
    elif k == 2:           # apply not operator in two random positions        
        y.solution[r[0]] = not(y.solution[r[0]])
        y.solution[r[1]] = not(y.solution[r[1]])
        
    elif k == 3:           # apply not operator in three random positions
        y.solution[r[0]] = not(y.solution[r[0]])
        y.solution[r[1]] = not(y.solution[r[1]])
        y.solution[r[2]] = not(y.solution[r[2]])        
    
    return y

# VND implemantation
def VND(x, k_max, probdata):

  k = 1
  while k <= k_max:
    # Encontra o melhor vizinho na vizinhança atual
    x_best = BestImprovementF1(x, probdata)
    # Muda a vizinhança
    x, k = neighborhoodChange(x, x_best, k)

  return x

def BestImprovementF1(x, probdata):
    x_best = x.copy()  # Cópia da solução inicial para guardar o melhor
    while True:
        improved = False
        for y in generate_neighborhood(x):
            if fobj_mindist(y, probdata) < fobj_mindist(x, probdata):
                x = y.copy()  # Atualiza a solução com a melhoria
                improved = True
                break  # Sai do loop interno se encontrar melhoria
        if not improved:
            break  # Sai do loop externo se não encontrar melhoria
    return x_best

def BestImprovementF2(x, probdata):
    x_best = x.copy()  # Cópia da solução inicial para guardar o melhor
    while True:
        improved = False
        for y in generate_neighborhood(x):
            if fobj_minPA(y, probdata) < fobj_minPA(x, probdata):
                x = y.copy()  # Atualiza a solução com a melhoria
                improved = True
                break  # Sai do loop interno se encontrar melhoria
        if not improved:
            break  # Sai do loop externo se não encontrar melhoria
    return x_best

def generate_neighborhood(x, k, probdata):
    # Gera uma vizinhança de soluções a partir da solução atual para um determinado valor de k.

    neighborhood = []

    # Gere os vizinhos aplicando shake com o mesmo valor de k
    for _ in range(probdata.num_neighbors):  # Determine o número de vizinhos a serem gerados
        neighbor = shake(x, k, probdata)
        neighborhood.append(neighbor)

    return neighborhood

# NeighborhoodChange implementation
def neighborhoodChange(x, xlinha, k):
    
    if xlinha.fitness_penalizado < x.fitness_penalizado:
        x = copy.deepcopy(xlinha)
        k  = 1
    else:
        k += 1
        
    return x, k

#### Refinamento(Busca Local)

In [ ]:
def avaliar_solucao(solucao_x, solucao_y, clientes):
    num_clientes, num_pontos_acesso = solucao_x.shape
    custo_total = 0

    # Calcular o custo total da solução
    for i in range(num_clientes):
        for j in range(num_pontos_acesso):
            custo_total += solucao_x[i, j] * clientes[j].consumo_banda * solucao_y[i]

    return custo_total

def busca_local(solucao_x, solucao_y, clientes, num_clientes, max_iter):
    melhor_solucao_x = np.copy(solucao_x)
    melhor_solucao_y = np.copy(solucao_y)
    melhor_avaliacao = avaliar_solucao(melhor_solucao_x, melhor_solucao_y)

    for _ in range(max_iter):
        # Faça uma perturbação na solução atual (por exemplo, trocando aleatoriamente alguns clientes entre pontos de acesso) chamar shake

        # Refine a solução perturbada usando uma heurística local (por exemplo, troca de clientes entre pontos de acesso para melhorar a solução)

        # Avalie a nova solução
        avaliacao = avaliar_solucao(solucao_x, solucao_y)

        # Atualize a melhor solução se a nova solução for melhor
        if avaliacao < melhor_avaliacao:
            melhor_solucao_x = np.copy(solucao_x)
            melhor_solucao_y = np.copy(solucao_y)
            melhor_avaliacao = avaliacao

    return melhor_solucao_x, melhor_solucao_y

#### VNS IMPLEMENTATION

In [ ]:
from time import time

def GVNS(x, l_max, k_max, t_max):
  ##  r_max: 
  ##  k_max: 
  ##  t_max: 
  start_time = time()
  current_solution = x
  best_solution = current_solution.copy()

  while True:
    k = 1
    while k <= k_max:
      # Gera uma solução perturbada
      perturbed_solution = shake(current_solution, k)

      # Aplica VND para refinar a solução perturbada
      improved_solution = VND(perturbed_solution.copy(), l_max)

      # Muda a vizinhança
      best_solution, k = neighborhoodChange(best_solution, improved_solution, k)

      # Verifica critério de parada por tempo
      if time() - start_time > t_max:
        return best_solution

    # Verifica critério de parada por iterações da vizinhança
    if k > k_max:
      break

  return best_solution